In [24]:
from bids import BIDSLayout
layout = BIDSLayout('/Users/linweihung/Documents/Python/BrainhackTaiwan/ds000228')
bold = layout.get(subject = 'pixar002', suffix = 'bold', extension = 'nii.gz')
bold

[<BIDSImageFile filename='/Users/linweihung/Documents/Python/BrainhackTaiwan/ds000228/sub-pixar002/func/sub-pixar002_task-pixar_bold.nii.gz'>]

In [25]:
md = bold[0].get_metadata()
md['RepetitionTime']

2

In [26]:
bold[0].get_entities()

{'datatype': 'func',
 'extension': '.nii.gz',
 'subject': 'pixar002',
 'suffix': 'bold',
 'task': 'pixar'}

In [27]:
layout.get(subject = 'pixar002')

[<BIDSJSONFile filename='/Users/linweihung/Documents/Python/BrainhackTaiwan/ds000228/sub-pixar002/anat/sub-pixar002_T1w.json'>,
 <BIDSImageFile filename='/Users/linweihung/Documents/Python/BrainhackTaiwan/ds000228/sub-pixar002/anat/sub-pixar002_T1w.nii.gz'>,
 <BIDSJSONFile filename='/Users/linweihung/Documents/Python/BrainhackTaiwan/ds000228/sub-pixar002/func/sub-pixar002_task-pixar_bold.json'>,
 <BIDSImageFile filename='/Users/linweihung/Documents/Python/BrainhackTaiwan/ds000228/sub-pixar002/func/sub-pixar002_task-pixar_bold.nii.gz'>]

In [ ]:
# Correct image headers from BIDS metadata
for bold_series in layout.get(suffix='bold', extension='nii.gz'):
    img = bold_series.get_image() #Returns nibabel image object
    zooms = img.header.get_zooms()
    zooms[3] = bold_series.get_metadata()['RepetitionTime']
    img.header.set_zooms(zooms)
    img.to_filename(bold_series)

In [29]:
layout.get_subjects()

['pixar001',
 'pixar002',
 'pixar003',
 'pixar004',
 'pixar005',
 'pixar006',
 'pixar007',
 'pixar008',
 'pixar009',
 'pixar010',
 'pixar011',
 'pixar012',
 'pixar013',
 'pixar014',
 'pixar015',
 'pixar016',
 'pixar017',
 'pixar018',
 'pixar019',
 'pixar020',
 'pixar021',
 'pixar022',
 'pixar023',
 'pixar024',
 'pixar025',
 'pixar026',
 'pixar027',
 'pixar028',
 'pixar029',
 'pixar030',
 'pixar031',
 'pixar032',
 'pixar033',
 'pixar034',
 'pixar035',
 'pixar036',
 'pixar037',
 'pixar038',
 'pixar039',
 'pixar040',
 'pixar041',
 'pixar042',
 'pixar043',
 'pixar044',
 'pixar045',
 'pixar046',
 'pixar047',
 'pixar048',
 'pixar049',
 'pixar050',
 'pixar051',
 'pixar052',
 'pixar053',
 'pixar054',
 'pixar055',
 'pixar056',
 'pixar057',
 'pixar058',
 'pixar059',
 'pixar060',
 'pixar061',
 'pixar062',
 'pixar063',
 'pixar064',
 'pixar065',
 'pixar066',
 'pixar067',
 'pixar068',
 'pixar069',
 'pixar070',
 'pixar071',
 'pixar072',
 'pixar073',
 'pixar074',
 'pixar075',
 'pixar076',
 'pixar077',

In [30]:
layout.get_tasks()

['pixar']

In [34]:
layout.get_suffixes(subject= bids.layout.Query.ANY)

['T1w', 'bold']

In [33]:
list(layout.get_entities(metadata=False))

['task', 'suffix', 'extension', 'subject', 'datatype']

In [36]:
bold_file = layout.get(subject = 'pixar001', suffix = 'bold',
                       extension = ['nii', 'nii.gz'])[0]
md = bold_file.get_metadata()
md.keys()

dict_keys(['CogAtlasID', 'EchoTime', 'EffectiveEchoSpacing', 'FlipAngle', 'MagneticFieldStrength', 'Manufacturer', 'ManufacturersModelName', 'PhaseEncodingDirection', 'RepetitionTime', 'SliceTiming', 'TaskName', 'dcmmeta_affine', 'dcmmeta_reorient_transform', 'dcmmeta_shape', 'dcmmeta_slice_dim', 'dcmmeta_version', 'global', 'time'])

In [37]:
md['EchoTime']

0.03

In [38]:
md['FlipAngle']

90

Many metadata terms are optional in BIDS, so queries that may work smoothly in one dataset, or on one file, may fail on others:

In [ ]:
md['SliceEncodingDirection']

In [40]:
layout.get_FlipAngles()

[7, 90, 89, 84, 88]

In [41]:
layout.get_suffix(FlipAngle = 7)

['T1w']

In [43]:
layout.get_suffix(FlipAngle = 88)

['bold']

In [44]:
layout.get_FlipAngle(suffix = 'bold')

[90, 89, 84, 88]

In [45]:
{fa: len(layout.get(FlipAngle = fa))
 for fa in layout.get_FlipAngle(suffix = 'bold')}

{90: 152, 89: 1, 84: 1, 88: 1}

In [46]:
layout.get_subjects(FlipAngle=88)

['pixar028']

In [53]:
layout = BIDSLayout('/Users/linweihung/Documents/Python/BrainhackTaiwan/ds000224')

In [54]:
participants = layout.get(suffix='participants',extension = 'tsv')[0]
participants

<BIDSDataFile filename='/Users/linweihung/Documents/Python/BrainhackTaiwan/ds000224/participants.tsv'>

In [55]:
participants.get_df()

,participant_id,gender,age,education_degree,education_years
0,sub-MSC01,M,34,Doctorate,22.0
1,sub-MSC02,M,34,Doctorate,28.0
2,sub-MSC03,F,29,Masters,18.0
3,sub-MSC04,F,28,Bachelors,22.0
4,sub-MSC05,M,27,Bachelors,20.0
5,sub-MSC06,F,24,Bachelors,17.5
6,sub-MSC07,F,31,Masters,20.0
7,sub-MSC08,F,27,Professional,21.0
8,sub-MSC09,M,26,Professional,19.0
9,sub-MSC10,M,31,Professional,19.0


In [56]:
collections = layout.get_collections(level='dataset')
collections

[<BIDSVariableCollection['age', 'education_degree', 'education_years', 'gender']>]

In [57]:
collections[0].to_df()

,subject,age,education_degree,education_years,gender,suffix
0,MSC01,34,Doctorate,22.0,M,participants
1,MSC02,34,Doctorate,28.0,M,participants
2,MSC03,29,Masters,18.0,F,participants
3,MSC04,28,Bachelors,22.0,F,participants
4,MSC05,27,Bachelors,20.0,M,participants
5,MSC06,24,Bachelors,17.5,F,participants
6,MSC07,31,Masters,20.0,F,participants
7,MSC08,27,Professional,21.0,F,participants
8,MSC09,26,Professional,19.0,M,participants
9,MSC10,31,Professional,19.0,M,participants


In [58]:
layout.get_collections(level = 'subject', subject = 'MSC01')[0].to_df()

,subject,session,acq_time,suffix
0,MSC01,func01,1894-08-25T01:07:51,sessions
1,MSC01,func02,1894-08-26T00:09:21,sessions
2,MSC01,func03,1894-08-27T00:17:24,sessions
3,MSC01,func04,1894-08-28T00:14:11,sessions
4,MSC01,func05,1894-08-29T00:08:33,sessions
5,MSC01,func06,1894-08-30T00:12:32,sessions
6,MSC01,func07,1894-08-31T00:08:55,sessions
7,MSC01,func08,1894-09-08T00:18:53,sessions
8,MSC01,func09,1894-09-09T00:15:21,sessions
9,MSC01,func10,1894-09-10T00:18:59,sessions


In [60]:
layout = BIDSLayout('/Users/linweihung/Documents/Python/BrainhackTaiwan/ds000224')
layout.get_collections(level = 'session', subject = 'MSC01', session = 'func01')[0].to_df()

,subject,session,run,BodyPartExamined,CogAtlasID,ConversionSoftware,ConversionSoftwareVersion,EchoTime,EffectiveEchoSpacing,FlipAngle,...,SequenceName,SequenceVariant,SeriesDescription,SliceTiming,SoftwareVersion,TaskName,acq_time,datatype,suffix,task
0,MSC01,func01,1.0,HEAD,http://www.cognitiveatlas.org/task/id/trm_4f24...,dcm2niix,v1.0.20170411 GCC4.4.7,0.027,0.00059,90,...,_epfid2d1_64,SK,MixedDesign_192Frames_36Slices_TR2.2,"[1.105, 0, 1.1675, 0.0625, 1.2275, 0.1225, 1.2...",syngo MR B17,glasslexical,1894-08-25T02:44:47,func,bold,glasslexical
1,MSC01,func01,2.0,HEAD,http://www.cognitiveatlas.org/task/id/trm_4f24...,dcm2niix,v1.0.20170411 GCC4.4.7,0.027,0.00059,90,...,_epfid2d1_64,SK,MixedDesign_192Frames_36Slices_TR2.2,"[1.105, 0, 1.1675, 0.0625, 1.2275, 0.1225, 1.2...",syngo MR B17,glasslexical,1894-08-25T02:53:17,func,bold,glasslexical


In [ ]:
layout.get_collections(level='run', subject='MSC01',session = 'func01',
                       task = 'motor', run=1)[0].to_df()

In [69]:
import json
layout = BIDSLayout('/Users/linweihung/Documents/Python/BrainhackTaiwan/ds000228')
entities = {"task": "rest", "suffix": "bold", "extension": "json"}
layout.build_path(entities)

'/Users/linweihung/Documents/Python/BrainhackTaiwan/ds000228/task-rest_bold.json'

In [77]:
layout.write_contents = json.dumps({"Instructions": "Just relax."})

In [ ]:
from bids.reports import BIDSReport
reports = BIDSReport(layout).generate()
print(reports.most_common()[0][0])

In [ ]:
import sys
from pathlib import Path
from bids import BIDSLayout
from bids.reports import BIDSReport

cmd, bids_root, out_root, analysis_level, *opts = sys.argv
layout = BIDSLayout(bids_root)

reports= {}
for subject in layout.get_subjects():
    rep = list(BIDSReport(layout).generate(subject = subject))[0]
    reports.setdefault(rep, []).append(subject)

with open(Path(out_root) / 'report.txt', 'w') as out_file:
    out_file.write(f"Scan parameter sets detected: {len(reports)}\n\n")
    for rep, subs in reports.items():
        rep = "\n".join(rep.splitlines()[:-2])
        out_file.write(f"-----/nParameters for subjects: {', '.join(subs)}\n\n")
        out_file.write(rep.replace('<deg>',"∘") + "\n\n")